In [10]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2

import coremltools

# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.layers.experimental import preprocessing

from tensorflow.keras.applications import MobileNetV3Large
from keras.layers import Dense, Dropout

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint



from tensorflow.keras import Model



In [2]:
TRAIN_DIR = '70-dog-breeds/train'
TEST_DIR = '70-dog-breeds/test'

train_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    validation_split = 0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

In [3]:
# Split the data into three categories.
train_images = train_generator.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_directory(
    TEST_DIR,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 6390 images belonging to 70 classes.
Found 1556 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [4]:
# Resize Layer
resize_and_rescale = keras.Sequential([
  preprocessing.Resizing(224,224),
  preprocessing.Rescaling(1./255),
])

# Setup data augmentation
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal_and_vertical"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),                       
], name="data_augmentation")

In [6]:
# Load the pretained model
pretrained_model = MobileNetV3Large(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

pretrained_model.trainable = False

print(len(train_images))
print(len(val_images))

200
49


In [12]:
# Build model
inputs = pretrained_model.input
x = resize_and_rescale(inputs)
x = data_augmentation(x)

x = Dense(512, activation='relu')(pretrained_model.output)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(70, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer = tf.keras.optimizers.legacy.Adam(0.0001),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [13]:
# Create checkpoint callback
checkpoint_path = "./Finish CNN-Model/Checkpoint"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)


# Create callback to visualize data
tb_callback = tf.keras.callbacks.TensorBoard(log_dir='./Finish CNN-Model/Graph', 
                                             histogram_freq=1, write_graph=True, 
                                             write_images=True, embeddings_freq=1)

In [14]:
# Train the model
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=val_images,
    validation_steps=len(val_images),
    epochs=50,
    callbacks=[
        EarlyStopping(patience = 5),
        tb_callback,
        checkpoint_callback,
    ]
)

Epoch 1/50
200/200 [==============================] - 41s 193ms/step - loss: 3.3345 - accuracy: 0.2340 - val_loss: 1.6901 - val_accuracy: 0.7114
Epoch 2/50
200/200 [==============================] - 40s 202ms/step - loss: 1.4209 - accuracy: 0.6599 - val_loss: 0.6661 - val_accuracy: 0.8400
Epoch 3/50
200/200 [==============================] - 39s 194ms/step - loss: 0.8272 - accuracy: 0.7806 - val_loss: 0.4990 - val_accuracy: 0.8605
Epoch 4/50
200/200 [==============================] - 39s 194ms/step - loss: 0.6285 - accuracy: 0.8219 - val_loss: 0.4281 - val_accuracy: 0.8760
Epoch 5/50
200/200 [==============================] - 39s 193ms/step - loss: 0.5200 - accuracy: 0.8474 - val_loss: 0.3918 - val_accuracy: 0.8760
Epoch 6/50
200/200 [==============================] - 39s 195ms/step - loss: 0.4418 - accuracy: 0.8726 - val_loss: 0.3816 - val_accuracy: 0.8811
Epoch 7/50
200/200 [==============================] - 39s 195ms/step - loss: 0.3893 - accuracy: 0.8890 - val_loss: 0.3690 - val_ac

In [16]:
# Show metrics
train_acc, train_loss = model.evaluate(train_images, verbose=0)
val_acc, val_loss = model.evaluate(val_images, verbose=0)
test_acc, test_loss = model.evaluate(test_images, verbose=0)
print("    Train Loss: {:.5f}".format(train_acc))
print("Train Accuracy: {:.2f}%".format(train_loss * 100))
print("    Valid Loss: {:.5f}".format(val_acc))
print("Valid Accuracy: {:.2f}%".format(val_loss * 100))
print("    Test Loss: {:.5f}".format(test_acc))
print("Test Accuracy: {:.2f}%".format(test_loss * 100))

    Train Loss: 0.03558
Train Accuracy: 99.20%
    Valid Loss: 0.33518
Valid Accuracy: 91.20%
    Test Loss: 0.10656
Test Accuracy: 97.00%


In [17]:
# Load the TensorBoard notebook extension
%reload_ext tensorboard